In [121]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("azminetoushikwasi/supplygraph-supply-chain-planning-using-gnns")

# print("Path to dataset files:", path)

In [122]:
import pandas as pd
path = "./supplygraph-supply-chain-planning-using-gnns/versions/2/Raw Dataset/"

# Nodes
nodes = pd.read_csv(path + "Nodes/Nodes.csv")   
edges = pd.read_csv(path + "Edges/Edges (Plant).csv")

delivery_to_distributor = pd.read_csv(path + "Temporal Data/Unit/Delivery To distributor.csv")
factory_issue = pd.read_csv(path + "Temporal Data/Unit/factory issue.csv")
production = pd.read_csv(path + "Temporal Data/Unit/Production .csv")
sales_order = pd.read_csv(path + "Temporal Data/Unit/Sales order.csv")



In [ ]:
import pandas as pd
import numpy as np
from torch_geometric_temporal.signal import StaticGraphTemporalSignal

path = "./supplygraph-supply-chain-planning-using-gnns/versions/2/Raw Dataset/Temporal Data/Unit/"

production = pd.read_csv(path + "Production .csv")
factory_issue = pd.read_csv(path + "factory issue.csv")
delivery = pd.read_csv(path + "Delivery To distributor.csv")
sales_order = pd.read_csv(path + "Sales order.csv")

products = [col for col in production.columns if col != "Date"]
product_to_id = {prod: i for i, prod in enumerate(products)}

def transform_temporal(df, mapping):
    df_no_date = df.drop(columns=["Date"])
    return df_no_date.rename(columns=mapping)

X_prod = transform_temporal(production, product_to_id)
X_issue = transform_temporal(factory_issue, product_to_id)
X_delivery = transform_temporal(delivery, product_to_id)
X_sales = transform_temporal(sales_order, product_to_id)  

X_prod_np = X_prod.to_numpy()
X_issue_np = X_issue.to_numpy()
X_delivery_np = X_delivery.to_numpy()
X_sales_np = X_sales.to_numpy()

# shape: [T, N, F] = [time_steps, num_nodes, num_features]
X = np.stack([X_prod_np, X_issue_np, X_delivery_np, X_sales_np], axis=-1)

y = X_sales_np[1:]       
X = X[:-1]

print("X shape:", X.shape)  # (T-1, N, 4)
print("y shape:", y.shape)  # (T-1, N)


X shape: (220, 41, 3)
y shape: (220, 41)


In [124]:
train = int(0.8 * X.shape[0])

X_train, X_test = X[:train], X[train:]
y_train, y_test = y[:train], y[train:]

In [125]:
from sklearn.preprocessing import StandardScaler

# Flatten for scaling: combine time and nodes
T, N, F = X_train.shape
X_flat = X_train.reshape(-1, F)  # shape [T*N, F]
y_flat = y_train.reshape(-1, 1)  # shape [T*N, 1]

# Create scalers
scaler_X = StandardScaler()
scaler_y = StandardScaler()

# Fit and transform
X_train_scaled_flat = scaler_X.fit_transform(X_flat)
y_train_scaled_flat = scaler_y.fit_transform(y_flat)

# Reshape back to original shape
X_train_scaled = X_train_scaled_flat.reshape(T, N, F)
y_train_scaled = y_train_scaled_flat.reshape(T, N)

print(f"X_scaled range: {X_train_scaled.min():.2f} to {X_train_scaled.max():.2f}")
print(f"y_scaled range: {y_train_scaled.min():.2f} to {y_train_scaled.max():.2f}")


X_scaled range: -0.42 to 8.37
y_scaled range: -0.35 to 11.07


In [126]:
y_test_flat = y_test.reshape(-1, 1)
y_test_scaled_flat = scaler_y.transform(y_test_flat)
y_test_scaled = y_test_scaled_flat.reshape(y_test.shape)

X_test_flat = X_test.reshape(-1, F)
X_test_scaled_flat = scaler_X.transform(X_test_flat)
X_test_scaled = X_test_scaled_flat.reshape(X_test.shape)

In [127]:
from sklearn.preprocessing import StandardScaler
import numpy as np

T, N, F = X_train.shape

# Prepare scaled containers
X_train_scaled = np.zeros_like(X_train)
y_train_scaled = np.zeros_like(y_train)
X_test_scaled = np.zeros_like(X_test)
y_test_scaled = np.zeros_like(y_test)

scaler_X_per_product = []
scaler_y_per_product = []

for node_idx in range(N):
    # Fit a scaler for features of this product
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()
    
    X_node = X_train[:, node_idx, :]  # (T, F)
    y_node = y_train[:, node_idx].reshape(-1, 1)
    
    X_train_scaled[:, node_idx, :] = scaler_X.fit_transform(X_node)
    y_train_scaled[:, node_idx] = scaler_y.fit_transform(y_node).ravel()
    
    # Scale test data using same scaler
    X_test_scaled[:, node_idx, :] = scaler_X.transform(X_test[:, node_idx, :])
    y_test_scaled[:, node_idx] = scaler_y.transform(y_test[:, node_idx].reshape(-1, 1)).ravel()
    
    scaler_X_per_product.append(scaler_X)
    scaler_y_per_product.append(scaler_y)


In [128]:
edges = pd.read_csv("./supplygraph-supply-chain-planning-using-gnns/versions/2/Raw Dataset/Edges/Edges (Plant).csv")

edges['node1'] = edges['node1'].astype(str)
edges['node2'] = edges['node2'].astype(str)

edges['node1'] = edges['node1'].map(product_to_id)
edges['node2'] = edges['node2'].map(product_to_id)

edge_index = edges[['node1', 'node2']].to_numpy().T.astype(np.int64)

num_nodes = len(product_to_id)

train_dataset = StaticGraphTemporalSignal(
    edge_index=edge_index,
    edge_weight=np.ones(edge_index.shape[1]),
    features=X_train_scaled,
    targets=y_train_scaled
)

test_dataset = StaticGraphTemporalSignal(
    edge_index=edge_index,
    edge_weight=np.ones(edge_index.shape[1]),
    features=X_test_scaled,
    targets=y_test_scaled
)


In [129]:
# from torch_geometric_temporal.signal import temporal_signal_split

# train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.7)

In [130]:
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import GCLSTM

class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features):
        super(RecurrentGCN, self).__init__()
        self.recurrent = GCLSTM(node_features, 32, 1)
        self.linear = torch.nn.Linear(32, 1)

    def forward(self, x, edge_index, edge_weight):
        h, c = self.recurrent(x, edge_index, edge_weight)  # Unpack (H, C)
        h = F.relu(h)
        h = self.linear(h)
        return h.squeeze(-1)

In [ ]:
model = RecurrentGCN(node_features=4)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [133]:
from tqdm import tqdm

def calculate_mse(model, dataset):
    """Calculate MSE on a dataset"""
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for time, snapshot in enumerate(dataset):
            y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
            total_loss += torch.mean((y_hat - snapshot.y) ** 2).item()
    model.train()
    return total_loss / (time + 1)

n_epoch = 100
for epoch in tqdm(range(n_epoch)):
    model.train()
    total_loss = 0
    for time, snapshot in enumerate(train_dataset):
        optimizer.zero_grad()
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
        loss = torch.mean((y_hat - snapshot.y) ** 2)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    if (epoch + 1) % 10 == 0:
        train_mse = calculate_mse(model, train_dataset)
        test_mse = calculate_mse(model, test_dataset)
        print(f"\nEpoch {epoch + 1}/{n_epoch}")
        print(f"Train MSE: {train_mse:.4f}")
        print(f"Test MSE: {test_mse:.4f}")


 10%|█         | 10/100 [00:10<01:43,  1.15s/it]


Epoch 10/100
Train MSE: 0.9739
Test MSE: 2.4134


 20%|██        | 20/100 [00:21<01:33,  1.17s/it]


Epoch 20/100
Train MSE: 0.9727
Test MSE: 2.4125


 30%|███       | 30/100 [00:32<01:20,  1.15s/it]

...


Epoch 80/100
Train MSE: 0.9659
Test MSE: 2.4127


 90%|█████████ | 90/100 [01:36<00:12,  1.21s/it]


Epoch 90/100
Train MSE: 0.9652
Test MSE: 2.4130


100%|██████████| 100/100 [01:48<00:00,  1.09s/it]


Epoch 100/100
Train MSE: 0.9650
Test MSE: 2.4128
